In [ ]:
import os
import sys
import datetime
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
datetime.datetime.now()

In [ ]:
ticker="GME"

In [ ]:
import mplfinance as mpf
from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries(key="REPLACE_ME", output_format="pandas")
df_stock, _ = ts.get_daily_adjusted(symbol=ticker, outputsize="compact")
df_stock.rename(columns = {'1. open':'Open', '2. high': 'High', '3. low':'Low', '4. close':'Close',
                           '5. adjusted close':'Adjusted Close','6. volume':'Volume'}, inplace = True)

In [ ]:
mpf.plot(df_stock.iloc[::-1],type='candle',mav=(10,20,50),volume=True, style='yahoo', figratio=(10,5), figscale=1.25)

In [ ]:
from gamestonk_terminal.due_diligence.finviz_api import analyst_df

In [ ]:
df_fa = analyst_df(ticker)
df_fa